In [1]:
import os
import re
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import tabulate
import json
import warnings

warnings.filterwarnings('ignore')

import data_preprocessing
import EDCR_pipeline
import vit_pipeline
import utils

# EDCR Results

In [2]:
def get_metrics(test_true: np.array, 
                prior_predictions: np.array, 
                post_predictions: np.array) -> dict:
    assert len(prior_predictions) == len(post_predictions)
    
    return {prior_or_post: ({'acc': accuracy_score(y_true=test_true, 
                                                       y_pred=(prior_predictions 
                                                       if prior_or_post == 'prior' else post_predictions))} | 
                                {metric_name: metric_value(y_true=test_true, 
                                                           y_pred=(prior_predictions 
                                                                   if prior_or_post == 'prior' else post_predictions), 
                                                           average='weighted') 
                                 for metric_name, metric_value in {'pre': precision_score, 'rec': recall_score, 'f1': f1_score}.items()})
                 for prior_or_post in ['prior', 'post']} | {'len': len(prior_predictions)}


def gather_EDCR_data() -> dict:
    data = {} 
    
    # Iterate through filenames to collect accuracy data
    for filename in os.listdir(EDCR_pipeline.figs_folder):
        secondary_granularity_match = re.match(
            pattern='main_(fine|coarse)_(.+?)_lr(.+?)_secondary_(fine|coarse)_(.+?)_lr(.+)',
            string=filename
        )
        
        if secondary_granularity_match:
            (   match,
                main_granularity,
                main_model_name,
                main_lr,
                secondary_granularity,
                secondary_model_name,
                secondary_lr
            ) = (secondary_granularity_match.group(i) for i in range(7))
            
            main_suffix = '_coarse' if main_granularity == 'coarse' else ''
            test_true = np.load(os.path.join(EDCR_pipeline.data_folder, f'test_true{main_suffix}.npy'))
            
            prior_predictions = np.load(os.path.join(EDCR_pipeline.data_folder, rf'{main_model_name}_test_pred_lr{main_lr}_e3{main_suffix}.npy'))
            
            secondary_suffix = '_coarse' if secondary_granularity == 'coarse' else ''
            post_predictions = np.load(f'figs/{match}/results{secondary_suffix}.npy')

            # Store accuracy data in the data dictionary
            if main_granularity not in data:
                data[main_granularity] = {}
            if main_model_name not in data[main_granularity]:
                data[main_granularity][main_model_name] = {}
            if secondary_granularity not in data[main_granularity][main_model_name]:
                data[main_granularity][main_model_name][secondary_granularity] = {}
            if secondary_model_name not in data[main_granularity][main_model_name][secondary_granularity]:
                data[main_granularity][main_model_name][secondary_granularity][secondary_model_name] = {}
            if main_lr not in data[main_granularity][main_model_name][secondary_granularity][secondary_model_name]:
                data[main_granularity][main_model_name][secondary_granularity][secondary_model_name][main_lr] = {}
                
            data[main_granularity][main_model_name][secondary_granularity][secondary_model_name][main_lr][secondary_lr] = get_metrics(test_true=test_true,
                                                                                                                                      prior_predictions=prior_predictions,
                                                                                                                                      post_predictions=post_predictions)
        else:
            no_secondary_granularity_match = re.match(pattern='main_(fine|coarse)_(.+)_lr(.+)_secondary_(.+)_lr(.+)',
                                                      string=filename)
            
            if no_secondary_granularity_match:
                
                (match,
                 main_granularity,
                 main_model_name,
                 main_lr,
                 secondary_model_name,
                 secondary_lr 
                ) = (no_secondary_granularity_match.group(i) for i in range(6))
                
                main_suffix = '_coarse' if main_granularity == 'coarse' else ''
                test_true = np.load(os.path.join(EDCR_pipeline.data_folder, f'test_true{main_suffix}.npy'))
                
                prior_predictions = np.load(os.path.join(EDCR_pipeline.data_folder, rf'{main_model_name}_test_pred_lr{main_lr}_e3{main_suffix}.npy'))
                
                try:
                    post_predictions = np.load(f'figs/{match}/results.npy')
                except FileNotFoundError:
                    post_predictions = np.load(f'figs/{match}/results_coarse.npy')
                    
                if main_granularity not in data:
                    data[main_granularity] = {}
                if main_model_name not in data[main_granularity]:
                    data[main_granularity][main_model_name] = {}
                if secondary_model_name not in data[main_granularity][main_model_name]:
                    data[main_granularity][main_model_name][secondary_model_name] = {}
                if main_lr not in data[main_granularity][main_model_name][secondary_model_name]:
                    data[main_granularity][main_model_name][secondary_model_name][main_lr] = {}
                
                data[main_granularity][main_model_name][secondary_model_name][main_lr][secondary_lr] = get_metrics(test_true=test_true,
                                                                                                                   prior_predictions=prior_predictions,
                                                                                                                   post_predictions=post_predictions)
    return data

def get_diff_str(diff: float) -> str:
    return f"({utils.green_text('+') if diff > 0 else ''}{(utils.green_text(f'{diff}%') if diff > 0  else utils.red_text(f'{diff}%'))})" if abs(diff) > 0 else ''

def get_row_addition(secondary_lr: float, 
                     curr_data: dict,
                     max_accuracy: float = None) -> (str, float):
    roundoff = 2
    curr_prior_data = curr_data['prior']
    curr_post_data = curr_data['post']
    
    curr_prior_accuracy = round(curr_prior_data['acc'] * 100, roundoff)
    curr_post_accuracy = round(curr_post_data['acc'] * 100, roundoff)
    curr_accuracy_diff = round(curr_post_accuracy - curr_prior_accuracy, roundoff)
    
    post_acc_str = (utils.blue_text(curr_post_accuracy) 
                    if max_accuracy is not None and abs(curr_post_accuracy - max_accuracy) < 1e-5 
                    else str(curr_post_accuracy))
    
    curr_prior_average_f1 = round(curr_prior_data['f1'] * 100, roundoff)
    curr_post_average_f1 = round(curr_post_data['f1'] * 100, roundoff)
    curr_average_f1_diff = round(curr_post_average_f1 - curr_prior_average_f1, roundoff)
    
    
    acc_str = f'acc: {post_acc_str}% {get_diff_str(curr_accuracy_diff)}'
    f1_str = f'f1: {curr_post_average_f1}% {get_diff_str(curr_average_f1_diff)}'
    row_addition = f"{secondary_lr}: {acc_str}, {f1_str}\n"

    return row_addition, curr_prior_accuracy, curr_prior_average_f1


def get_row_data(main_lr_data: dict,
                 secondary_lr: float) -> (str, float):
    curr_data = main_lr_data[secondary_lr]
    row_addition, curr_prior_acc, curr_prior_average_f1 = get_row_addition(secondary_lr=secondary_lr, 
                                                                                   curr_data=curr_data)
    
    return row_addition, curr_prior_acc, curr_prior_average_f1


def print_one_secondary_granularity(main_model_data: dict,
                                    secondary_granularity: str,
                                    main_granularity: str,
                                    main_model_name: str):

    secondary_granularity_data = main_model_data[secondary_granularity]
    main_learning_rates = sorted(secondary_granularity_data[list(secondary_granularity_data.keys())[0]].keys())
    header = [''] + main_learning_rates
    table_data = [header]
    priors = {}

    for secondary_model_name in sorted(secondary_granularity_data.keys()):
        secondary_model_data = secondary_granularity_data[secondary_model_name]
        row = [secondary_model_name]
        
        for main_lr in sorted(secondary_model_data.keys()):
            main_lr_data = secondary_model_data[main_lr]
            row_add = ''
            
            for secondary_lr in sorted(main_lr_data.keys()):
                row_addition, curr_prior_acc, curr_prior_average_f1 = get_row_data(main_lr_data=main_lr_data,
                                                                                      secondary_lr=secondary_lr)
                row_add += row_addition
                priors[main_lr] = {'acc': curr_prior_acc, 'f1': curr_prior_average_f1}
                
            row += [row_add]
        table_data += [row]
    
    table_data[0] = [''] + [f"main-lr={main_lr} (acc: {priors[main_lr]['acc']}%, f1: {priors[main_lr]['f1']}%)" for main_lr in main_learning_rates]
    
    # Rest of your code to create and print the table remains unchanged
    table = tabulate.tabulate(
        tabular_data=table_data, 
        headers='firstrow', 
        tablefmt='grid'
    )
    print(f"Main model: {main_granularity.capitalize()}-grain {main_model_name}, "
          f"secondary granularity: {secondary_granularity}")
    print(table)
    print("\n")


def print_two_secondary_granularities(main_model_data: dict,
                                      two_secondary_table_data: list,
                                      secondary_model_name: str,
                                      main_granularity: str,
                                      main_model_name: str):
    main_learning_rates = sorted(vit_pipeline.lrs)
    
    priors = {}
    
    # Initialize the table_data with header if it's empty
    if len(two_secondary_table_data) == 0:
        header = [''] + main_learning_rates
        two_secondary_table_data += [header]
        
    secondary_model_data = main_model_data[secondary_model_name]
    row = [secondary_model_name]
    
    for main_lr in sorted(secondary_model_data.keys()):
        main_lr_data = secondary_model_data[main_lr]
        row_add = ''
        
        for secondary_lr in sorted(main_lr_data.keys()):
            row_addition, curr_prior_acc, curr_prior_average_f1 = get_row_data(main_lr_data=main_lr_data,
                                                    secondary_lr=secondary_lr)
            row_add += row_addition
            priors[main_lr] = {'acc': curr_prior_acc, 'f1': curr_prior_average_f1}
    
        row += [row_add]

    two_secondary_table_data += [row]
    
    # Modify the generated table data to highlight the cell with the maximal accuracy in blue
    
    if len(two_secondary_table_data) == len(main_learning_rates) + 1:
        two_secondary_table_data[0] = [''] + [f"main-lr={main_lr} (acc: {priors[str(main_lr)]['acc']}%, f1: {priors[str(main_lr)]['f1']}%)" for main_lr in main_learning_rates]
        
        # Create the table using tabulate
        table = tabulate.tabulate(
            tabular_data=two_secondary_table_data,
            headers='firstrow',
            tablefmt='grid'
        )
        
        # Print the main model name and the corresponding table
        print(f"Main model: {main_granularity.capitalize()}-grain {main_model_name} "
              f"with both fine and coarse grain secondary models")
        print(table)
        print("\n")
    else:
        return two_secondary_table_data


def print_EDCR_tables():
    data = gather_EDCR_data()
    
    for main_granularity in sorted(data.keys()):
        
        print('#' * 40 + f' Main granularity: {main_granularity} ' + '#' * 40 + '\n' + '#' * 104 + '\n')
        main_granularity_data = data[main_granularity]
        
        for main_model_name in sorted(main_granularity_data.keys()):
            main_model_data = main_granularity_data[main_model_name]
            two_secondary_table_data = []

            for granularity_or_model in (sorted(set(main_model_data.keys()).intersection(data_preprocessing.granularities.values())) + 
                      sorted(set(main_model_data.keys()).intersection(vit_pipeline.vit_model_names))):
            
                if granularity_or_model in data_preprocessing.granularities.values():
                    print_one_secondary_granularity(main_model_data=main_model_data,
                                                    secondary_granularity=granularity_or_model,
                                                    main_granularity=main_granularity,
                                                    main_model_name=main_model_name)
                else:
                    two_secondary_table_data = print_two_secondary_granularities(main_model_data=main_model_data,
                                                                                 two_secondary_table_data=two_secondary_table_data,
                                                                                 secondary_model_name=granularity_or_model,
                                                                                 main_granularity=main_granularity,
                                                                                 main_model_name=main_model_name)
            print('#' * 100)

print_EDCR_tables()

In [3]:
best_coarse_main_model = 'vit_l_16'
best_coarse_main_lr = '1e-06'
best_coarse_secondary_model = 'vit_b_16'
best_coarse_secondary_lr = '5e-05'
best_coarse_folder = f'main_coarse_{best_coarse_main_model}_lr{best_coarse_main_lr}_secondary_{best_coarse_secondary_model}_lr{best_coarse_secondary_lr}'
best_coarse_results = np.load(rf'{EDCR_pipeline.figs_folder}{best_coarse_folder}/results.npy')
coarse_test_true = np.load(os.path.join(EDCR_pipeline.data_folder, f'test_true_coarse.npy'))

best_fine_main_model = 'vit_l_16'
best_fine_main_lr = '1e-06'
best_fine_secondary_model = 'vit_b_16'
best_fine_secondary_lr = '1e-06'
best_fine_folder = f'main_fine_{best_fine_main_model}_lr{best_fine_main_lr}_secondary_{best_fine_secondary_model}_lr{best_fine_secondary_lr}'
best_fine_results = np.load(rf'{EDCR_pipeline.figs_folder}{best_fine_folder}/results.npy')
fine_test_true = np.load(os.path.join(EDCR_pipeline.data_folder, f'test_true.npy'))

with open('fine_to_coarse.json', 'r') as json_file:
    image_fine_to_coarse = json.load(json_file)

image_fine_to_coarse = {int(fine): int(coarse) for batch_dict in image_fine_to_coarse for fine, coarse in batch_dict.items()}
image_fine_to_coarse

# def get_num_of_inconsistencies(coarse_results: np.array, 
#                                fine_results: np.array) -> int:
#     num_of_inconsistencies = 0
#     for fine_example_num, fine_prediction_index in enumerate(fine_results):
#         coarse_example_num = image_fine_to_coarse[fine_example_num]
#         coarse_prediction_index = coarse_results[coarse_example_num]
#         
#         fine_prediction = EDCR_pipeline.get_classes(granularity='fine')[fine_prediction_index]
#         coarse_prediction = EDCR_pipeline.get_classes(granularity='coarse')[coarse_prediction_index]
#         derived_coarse_prediction = EDCR_pipeline.fine_to_coarse[fine_prediction]
# 
#         if derived_coarse_prediction != coarse_prediction:
#             num_of_inconsistencies += 1
# 
#     return num_of_inconsistencies
# 
# get_num_of_inconsistencies(coarse_results=coarse_test_true,
#                            fine_results=fine_test_true)

In [9]:
best_coarse_main_model = 'vit_l_16'
best_coarse_main_lr = '1e-06'
best_coarse_secondary_model = 'vit_b_16'
best_coarse_secondary_lr = '5e-05'

folder = (f'{EDCR_pipeline.figs_folder}/main_coarse_{best_coarse_main_model}_lr{best_coarse_main_lr}'
                  f'_secondary_{best_coarse_secondary_model}_lr{best_coarse_secondary_lr}')

with open(f'{folder}/error_detections.json', 'r') as json_file:
    error_detections = json.load(json_file)

print('Error detections:\n')
for coarse_grain_label, coarse_grain_label_data in error_detections.items():
    for fine_grain_label in coarse_grain_label_data.keys():
        print(f'error <- predicted_coarse_grain = {coarse_grain_label} '
              f'and predicted_fine_grain = {fine_grain_label}')

In [14]:
with open(f'{folder}/corrections.json', 'r') as json_file:
    corrections = json.load(json_file)

print('Corrections:\n')
for coarse_grain_label, coarse_grain_label_data in corrections.items():
    for fine_grain_label in coarse_grain_label_data.keys():
        print(f'correct_coarse_grain = {EDCR_pipeline.fine_to_coarse[fine_grain_label]} <- predicted_coarse_grain = {coarse_grain_label} '
              f'and predicted_fine_grain = {fine_grain_label}')